In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import sys

# Option 1: Utilisation directe de l'API REST

def api_direct_access():
    """Exemple d'accès direct à l'API de Statistique Canada"""
    print("Méthode 1: Accès direct à l'API REST")
    
    base_url = "https://www150.statcan.gc.ca/t1/wds/rest/"
    product_id = "18100245"  # ID du tableau sur les prix de détail moyens pour certains produits alimentaires
    
    # Étape 1: Obtenir les métadonnées du tableau
    endpoint = base_url + "getCubeMetadata"
    payload = [{"productId": product_id}]
    headers = {"Content-Type": "application/json"}
    
    try:
        response = requests.post(endpoint, data=json.dumps(payload), headers=headers)
        metadata = response.json()
        
        print(f"Métadonnées du tableau {product_id} récupérées.")
        print(f"Titre: {metadata[0].get('object', {}).get('cubeTitleEn', 'Non disponible')}")
        
        # Extraire quelques vecteurs (IDs de séries) pour les utiliser dans la prochaine étape
        vectors = []
        if 'object' in metadata and 'dimension' in metadata['object']:
            # Logique pour extraire les vecteurs - cela dépend de la structure exacte des métadonnées
            print("Structure des dimensions:")
            for dim in metadata['object']['dimension']:
                print(f"- {dim.get('dimensionNameFr', 'N/A')}")
                
        print(f"\nPour accéder aux données complètes, vous devriez explorer les métadonnées pour identifier les vecteurs spécifiques qui vous intéressent.")
        
        # Étape 2: Exemple d'extraction de données pour un vecteur spécifique
        # Notez: Vous devrez remplacer "VECTOR_ID" par un vrai ID de vecteur du tableau
        print("\nExemple d'extraction de données pour un vecteur spécifique (nécessite un ID de vecteur valide):")
        print("endpoint = base_url + 'getDataFromVectorsAndLatestNPeriods'")
        print("payload = {'vectorIds': ['VECTOR_ID'], 'latestN': 5}")
        
        return metadata
        
    except Exception as e:
        print(f"Une erreur s'est produite: {e}")
        return None

In [9]:
def get_all_cubes_list():
    """Exemple d'accès à la liste complète des cubes via l'API REST"""
    print("Méthode: Obtenir la liste complète des cubes")
    
    base_url = "https://www150.statcan.gc.ca/t1/wds/rest/"
    endpoint = base_url + "getAllCubesList"
    
    try:
        response = requests.get(endpoint)
        cubes_list = response.json()
        
        print("Liste complète des cubes récupérée.")
        return cubes_list
        
    except Exception as e:
        print(f"Une erreur s'est produite: {e}")
        return None

# Appeler la fonction pour obtenir la liste des cubes
cubes_list = get_all_cubes_list()

Méthode: Obtenir la liste complète des cubes
Liste complète des cubes récupérée.


In [10]:
cubes_list

[{'productId': 10100001,
  'cansimId': '183-0021',
  'cubeTitleEn': 'Federal public sector employment reconciliation of Treasury Board of Canada Secretariat, Public Service Commission of Canada and Statistics Canada statistical universes, as at December 31',
  'cubeTitleFr': 'Emploi du secteur public fédéral rapprochement des univers statistiques du Secrétariat du Conseil du Trésor du Canada, de la Commission de la fonction publique du Canada et de Statistique Canada, au 31 décembre',
  'cubeStartDate': '1999-01-01T05:00:00Z',
  'cubeEndDate': '2011-01-01T05:00:00Z',
  'releaseTime': '2012-08-01T12:30:00Z',
  'archived': '1',
  'subjectCode': ['10'],
  'surveyCode': ['1713'],
  'frequencyCode': 12,
  'corrections': [],
  'issueDate': '2012-08-01T04:00:00Z',
  'dimensions': [{'dimensionNameEn': 'Geography',
    'dimensionNameFr': 'Géographie',
    'dimensionPositionId': 1,
    'hasUOM': False},
   {'dimensionNameEn': 'Federal public sector employment',
    'dimensionNameFr': "Groupes d'

In [16]:
metadata = api_direct_access()

Méthode 1: Accès direct à l'API REST
Métadonnées du tableau 18100245 récupérées.
Titre: Monthly average retail prices for selected products

Pour accéder aux données complètes, vous devriez explorer les métadonnées pour identifier les vecteurs spécifiques qui vous intéressent.

Exemple d'extraction de données pour un vecteur spécifique (nécessite un ID de vecteur valide):
endpoint = base_url + 'getDataFromVectorsAndLatestNPeriods'
payload = {'vectorIds': ['VECTOR_ID'], 'latestN': 5}


In [5]:
def download_csv():
    """Download the full table as a CSV file."""
    url = "https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/18100245/fr"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            file_to_download = response.json()["object"]
            # Download the zip file

            response = requests.get(file_to_download)
            if response.status_code == 200:
                with open("table.zip", "wb") as file:
                    file.write(response.content)
                print("ZIP file downloaded successfully.")

                # Unzip the file
                import zipfile
                with zipfile.ZipFile("table.zip", "r") as zip_ref:
                    zip_ref.extractall("table")
                print("ZIP file extracted successfully.")
            else:
                print(f"Failed to download CSV. Status code: {response.status_code}")
        else:
            print(f"Failed to download CSV. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function
download_csv()

ZIP file downloaded successfully.
ZIP file extracted successfully.


In [7]:
# load table/18100245.csv into a pandas dataframe
def load_csv_to_dataframe():
    """Load the CSV file into a pandas DataFrame."""
    try:
        df = pd.read_csv("table/18100245.csv", sep=";", encoding="utf-8")
        print("CSV file loaded into DataFrame successfully.")
        return df
    except Exception as e:
        print(f"An error occurred while loading the CSV file: {e}")
        return None
    
df = load_csv_to_dataframe()

CSV file loaded into DataFrame successfully.


In [20]:
# filter GÉO == "Quebec" and PÉRIODE DE RÉFÉRENCE == "2025-03"
df_f = df[(df["GÉO"] == "Québec") & (df["PÉRIODE DE RÉFÉRENCE"] == "2025-03")].copy()
# split text in Produits column over "," and make them 2 columns and add them to the dataframe
df_f[["Produit 1", "Produit 2"]] = df_f["Produits"].str.split(",", n=2, expand=True)[[0, 1]]
df_f

,PÉRIODE DE RÉFÉRENCE,GÉO,DGUID,Produits,UNITÉ DE MESURE,IDENTIFICATEUR D'UNITÉ DE MESURE,FACTEUR SCALAIRE,IDENTIFICATEUR SCALAIRE,VECTEUR,COORDONNÉES,VALEUR,STATUS,SYMBOLE,TERMINÉ,DÉCIMALES,Produit 1,Produit 2
119031,2025-03,Québec,2016A000224,"Boeuf à ragoût, par kilogramme",Dollars,81,unités,0,v1159447135,5.100,17.79,NaN,NaN,NaN,2,Boeuf à ragoût,par kilogramme
119032,2025-03,Québec,2016A000224,"Contrefilets de boeuf, par kilogramme",Dollars,81,unités,0,v1159447136,5.200,33.29,NaN,NaN,NaN,2,Contrefilets de boeuf,par kilogramme
119033,2025-03,Québec,2016A000224,"Hauts de surlonge de boeuf, par kilogramme",Dollars,81,unités,0,v1159447137,5.300,19.15,NaN,NaN,NaN,2,Hauts de surlonge de boeuf,par kilogramme
119034,2025-03,Québec,2016A000224,"Côtes de boeuf, par kilogramme",Dollars,81,unités,0,v1353834495,5.410,50.91,NaN,NaN,NaN,2,Côtes de boeuf,par kilogramme
119035,2025-03,Québec,2016A000224,"Boeuf haché, par kilogramme",Dollars,81,unités,0,v1159447138,5.400,13.94,NaN,NaN,NaN,2,Boeuf haché,par kilogramme
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119136,2025-03,Québec,2016A000224,"Graines de tournesol, 400 grammes",Dollars,81,unités,0,v1458870100,5.109,5.10,NaN,NaN,NaN,2,Graines de tournesol,400 grammes
119137,2025-03,Québec,2016A000224,"Déodorant, 85 grammes",Dollars,81,unités,0,v1353834529,5.750,6.96,NaN,NaN,NaN,2,Déodorant,85 grammes
119138,2025-03,Québec,2016A000224,"Dentifrice, 100 millilitres",Dollars,81,unités,0,v1353834530,5.760,3.65,NaN,NaN,NaN,2,Dentifrice,100 millilitres
119139,2025-03,Québec,2016A000224,"Shampooing, 400 millilitres",Dollars,81,unités,0,v1353834531,5.770,6.70,NaN,NaN,NaN,2,Shampooing,400 millilitres


In [18]:
df_f["Produits"].str.split(",", n=2, expand=True)[[0, 1]]

,0,1
119031,Boeuf à ragoût,par kilogramme
119032,Contrefilets de boeuf,par kilogramme
119033,Hauts de surlonge de boeuf,par kilogramme
119034,Côtes de boeuf,par kilogramme
119035,Boeuf haché,par kilogramme
...,...,...
119136,Graines de tournesol,400 grammes
119137,Déodorant,85 grammes
119138,Dentifrice,100 millilitres
119139,Shampooing,400 millilitres


In [19]:
metadata

[{'status': 'SUCCESS',
  'object': {'responseStatusCode': 0,
   'productId': '18100245',
   'cansimId': None,
   'cubeTitleEn': 'Monthly average retail prices for selected products',
   'cubeTitleFr': 'Prix de détail moyens mensuels pour certains produits',
   'cubeStartDate': '2017-01-01',
   'cubeEndDate': '2025-03-01',
   'frequencyCode': 6,
   'nbSeriesCube': 1209,
   'nbDatapointsCube': 119691,
   'releaseTime': '2025-05-08T08:30',
   'archiveStatusCode': '2',
   'archiveStatusEn': 'CURRENT - a cube available to the public and that is current',
   'archiveStatusFr': 'ACTIF - un cube qui est disponible au public et qui est toujours mise a jour',
   'subjectCode': ['1802', '2001'],
   'surveyCode': ['2301'],
   'dimension': [{'dimensionPositionId': 1,
     'dimensionNameEn': 'Geography',
     'dimensionNameFr': 'Géographie',
     'hasUom': False,
     'member': [{'memberId': 11,
       'parentMemberId': None,
       'memberNameEn': 'Canada',
       'memberNameFr': 'Canada',
       '

In [2]:
!pip install matplotlib

     ---------------------------------------- 7.8/7.8 MB 55.4 MB/s eta 0:00:00
     ---------------------------------------- 111.1/111.1 KB ? eta 0:00:00
     ------------------------------------- 211.8/211.8 KB 12.6 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 146.3 MB/s eta 0:00:00
     ---------------------------------------- 55.8/55.8 KB ? eta 0:00:00


You should consider upgrading via the 'H:\GitHub\general-python-coding\venv\Scripts\python.exe -m pip install --upgrade pip' command.
